In [3]:
import pandas as pd
pd.read_excel('SanatorioGüemes/MachLearning_Sectores_de_internacion.xlsx')

,CodSector,NombreSector,TipoSector
0,ACV,UNIDAD DE ACV,Unidad de ACV
1,CEG,CUIDADOS ESPEC. EN OBSERVACION,Cuidados especiales Guardia
2,CO3,CORDOBA 3 (INT.GRAL),CORDOBA - Internacion General
3,CO4,CORDOBA 4 (INT.GRAL),CORDOBA - Internacion General
4,CO5,CORDOBA 5 (INT.GRAL),CORDOBA - Internacion General
5,CO6,CORDOBA 6 (INT.GRAL),CORDOBA - Internacion General
6,CO7,CORDOBA 7 (INT. GRAL),CORDOBA - Internacion General
7,CO8,CORDOBA 8 (INT. GRAL),CORDOBA - Internacion General
8,EME,EMERGENCIAS - CEG,Emergencias
9,H10,HEMOTERAPIA 4 SS TORRE,Hemoterapia


In [5]:
internaciones = pd.read_excel('SanatorioGüemes/MachLearning_Internaciones_ene2017_11nov2019.xlsx')

In [16]:
#pd.set_option('display.max_colwidth', -1)

pd.set_option('display.max_columns', None)  
internaciones.head()

,Nro Adm,Nro H.C.,Edad,Sexo,Fec Nac,Entidad,AgrupEntidad,Fec Adm,Hora Adm,AñoAdm,MesAdm,Procedencia,Médico Admisión,SecAdmisión,SecUltimo,CategUlt,Aislación,HabitUlt,CamaUlt,Fec Alta,Hora Alta,AñoAlta,MesAlta,Motivo Alta,PreAlta,1erSecInt,Diagnóstico Administrativo No Codificado,CodDiagPresu,Diagnóstico Presuntivo,CodCieDiagEgr,Diagnóstico Egreso,FecDenunEgre,HoraDenunEgre,Médico Denuncia Egreso,Epicrisis,MédicoEpicrisis,Quirurg,CaderaExpress,Permanencia,Servicio Responsable,Servicio Co-Responsable,FecHorIngGua,Prest.Guardia,Prestac.Guardia,Reingreso,AdmAntReciente,FecAdmAntReciente,FecAltaAntReciente,SecAntReciente,Diagn. Egreso Admisión Anterior Reciente,AmbulanciaEgreso,PesoAlNacer,EdadGestac,PIM2TEP,DiagAltoRiesgoTEP,DiagBajoRiesgoTEP,ARM_TEP,CEC_TEP,SolicDerivación,OrigDerivación,Procedencia.1,DiagnósticoDerivación,AreaDerivación,Notificado,UsuarioDeriv
0,466470-1,3526141-4,79,F,1937-11-15,PAMI UGL VI,PAMI,2017-01-01,02:42:00,2017,Ene,Emergencias,VEDIA VARGAS ANICETA,EME,T16,C,NaN,1634,2,2017-01-04,18:17:00,2017.0,Ene,Alta Medica,NaT,T16,DESHIDRATACION,XX2,No Especifica Codigo,276.51,Deshidratacion,2017-01-04,16:47:41,SIMONE MILENA,Sí,SIMONE MILENA,No,No,3,CLINICA MEDICA,NaN,NaN,NaN,NaN,No,NaN,NaT,NaT,NaN,NaN,Sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,466471-4,3714597-2,74,F,1942-09-13,PAMI UGL VI,PAMI,2017-01-01,02:48:00,2017,Ene,Emergencias,VEDIA VARGAS ANICETA,EME,UC6,A,*,702,1,2017-01-07,14:03:00,2017.0,Ene,Alta Medica,NaT,UCO,DIF RESPIRATORIA,XX2,No Especifica Codigo,401.9,Hipertension Arterial No Especificada,2017-01-07,13:42:01,GIARROCCO AGUSTINA,Sí,GIARROCCO AGUSTINA,No,No,6,CARDIOLOGIA ADULTOS,NaN,NaN,NaN,NaN,No,NaN,NaT,NaT,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,466472-7,3714595-2,27,F,1989-11-08,MEDICUS,PREPAGA,2017-01-01,02:46:00,2017,Ene,Guardia,QUIROZ CARINA MAGDALENA,T10,T10,C,NaN,1020,1,2017-01-01,19:54:00,2017.0,Ene,Alta Medica,NaT,T10,ABORTO ESPONTANEO,XX2,No Especifica Codigo,634,Aborto Espontaneo,2017-01-01,19:30:56,ABADIA MICAELA,Sí,ABADIA MICAELA,Sí,No,1,OBSTETRICIA,NaN,NaN,NaN,NaN,No,NaN,NaT,NaT,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,466473-0,3672378-8,85,F,1931-08-09,OSUTHGRA O.S PERSONAL GASTRONOMICO,BRISTOL,2017-01-01,03:55:00,2017,Ene,Emergencias,VEDIA VARGAS ANICETA,EME,T12,C,NaN,1236,1,2017-01-04,14:58:00,2017.0,Ene,Alta Medica,NaT,T12,OCLUSION INTESTINAL,789.00,Dolor Abdominal Sitio Inespecificado,620.2,Otros Quistes Ovaricos Y Quister Ovaricos Neom,2017-01-04,14:25:44,CHAVEZ MORA JORGE MARIO,Sí,CHAVEZ MORA JORGE MARIO,No,No,3,CLINICA MEDICA,NaN,NaN,NaN,NaN,No,NaN,NaT,NaT,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42736.059525,MESA OPERATIVA,DOM,OCLUSION INTESTINAL,INTERNACION GENERAL,SUP,ABLANCO
4,466474-3,3714598-2,34,F,1982-04-06,OSDE,PREPAGA,2017-01-01,05:07:00,2017,Ene,Emergencias,CANIS DIEGO DAMIAN,EME,EME,I,NaN,6,1,2017-01-01,09:01:00,2017.0,Ene,Alta Medica,NaT,EME,INTOX. ALCOHOLICA,XX2,No Especifica Codigo,94.68,Desintoxicacion Combinada De Alcohol Y Drogas,2017-01-01,09:01:03,PUENTE MOSQUERA KAROLA,Sí,PUENTE MOSQUERA KAROLA,No,No,1,NaN,NaN,NaN,NaN,NaN,No,NaN,NaT,NaT,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42736.195347,ANDREA,DOMICILIO,INTOX. ALCOHOLICA,INTERNACION GENERAL,NaN,ABLANCO


In [9]:
internaciones.columns

Index(['Nro Adm', 'Nro H.C.', 'Edad', 'Sexo', 'Fec Nac', 'Entidad',
       'AgrupEntidad', 'Fec Adm', 'Hora Adm', 'AñoAdm', 'MesAdm',
       'Procedencia', 'Médico Admisión', 'SecAdmisión', 'SecUltimo',
       'CategUlt', 'Aislación', 'HabitUlt', 'CamaUlt', 'Fec Alta', 'Hora Alta',
       'AñoAlta', 'MesAlta', 'Motivo Alta', 'PreAlta', '1erSecInt',
       'Diagnóstico Administrativo No Codificado', 'CodDiagPresu',
       'Diagnóstico Presuntivo', 'CodCieDiagEgr', 'Diagnóstico Egreso',
       'FecDenunEgre', 'HoraDenunEgre', 'Médico Denuncia Egreso', 'Epicrisis',
       'MédicoEpicrisis', 'Quirurg', 'CaderaExpress', 'Permanencia',
       'Servicio Responsable', 'Servicio Co-Responsable', 'FecHorIngGua',
       'Prest.Guardia', 'Prestac.Guardia', 'Reingreso', 'AdmAntReciente',
       'FecAdmAntReciente', 'FecAltaAntReciente', 'SecAntReciente',
       'Diagn. Egreso Admisión Anterior Reciente', 'AmbulanciaEgreso',
       'PesoAlNacer', 'EdadGestac', 'PIM2TEP', 'DiagAltoRiesgoTEP',
       

In [12]:
internaciones.describe()

,Edad,AñoAdm,CamaUlt,AñoAlta,Permanencia,FecHorIngGua,Prest.Guardia,PesoAlNacer,EdadGestac,PIM2TEP,DiagAltoRiesgoTEP,DiagBajoRiesgoTEP,SolicDerivación
count,88256.000000,88256.000000,88256.000000,87720.000000,88256.000000,21193.000000,2.119300e+04,5511.000000,5392.000000,440.000000,440.000000,440.000000,20734.000000
mean,45.231588,2017.983571,2.041028,2017.986651,6.193619,43360.379525,4.201101e+07,3176.442168,36.817136,3.081136,0.327273,0.104545,43306.854615
std,27.427388,0.806299,3.631540,0.805575,16.743699,249.242133,3.022844e+04,802.857439,7.958221,7.016981,1.424543,0.413857,288.622084
min,0.000000,2017.000000,1.000000,2017.000000,1.000000,42917.005683,4.201010e+07,0.000000,0.000000,0.000000,0.000000,0.000000,42736.059525
25%,25.000000,2017.000000,1.000000,2017.000000,1.000000,43150.589340,4.201010e+07,2920.000000,38.000000,0.300000,0.000000,0.000000,43070.785602
50%,45.000000,2018.000000,1.000000,2018.000000,2.000000,43369.032280,4.201010e+07,3310.000000,39.000000,1.500000,0.000000,0.000000,43330.214201
75%,68.000000,2019.000000,2.000000,2019.000000,5.000000,43577.510602,4.201011e+07,3650.000000,40.000000,3.725000,0.000000,0.000000,43550.344592
max,107.000000,2019.000000,32.000000,2019.000000,896.000000,43779.943137,4.302010e+07,5530.000000,71.000000,92.600000,9.000000,2.000000,43780.982812


In [14]:
internaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88256 entries, 0 to 88255
Data columns (total 65 columns):
Nro Adm                                     88256 non-null object
Nro H.C.                                    88256 non-null object
Edad                                        88256 non-null int64
Sexo                                        88256 non-null object
Fec Nac                                     88245 non-null datetime64[ns]
Entidad                                     88256 non-null object
AgrupEntidad                                88256 non-null object
Fec Adm                                     88256 non-null datetime64[ns]
Hora Adm                                    88256 non-null object
AñoAdm                                      88256 non-null int64
MesAdm                                      88256 non-null object
Procedencia                                 88251 non-null object
Médico Admisión                             86786 non-null object
SecAdmisión        

In [18]:
images= pd.read_excel('SanatorioGüemes/MachLearning_Estudios_Imagenes_solicitados.xlsx')

In [19]:
images.head()

,Nro Vale,Pun,Estado,Fecha,Hora,HH,AñoMes,Sector,Nro Adm,HistClín,Fec.Adm.,Fec.Alta,Entidad,Nombre Entidad,GrupoEnt,Urgencia,Mnemo Serv,Servicio,Cod Serv,Nombre del Solicitante,Función del Solicitante,Prestación,Descrip Prestación,CantPrest,Cod Insumo,Descrip Insumo,CantInsumos,Observ.Estudio
0,35455089,35454865,3,2017-01-01,03:20:00,3,2017-Ene,EME,466471-4,3714597-2,2017-01-01,2017-01-07,876,PAMI UGL VI,PAMI,1,RX,RAYOS,7100,VEDIA ANICETA,MEDICO DE PISO,34090503,RX C/PORTATIL DE TORAX,1,NaN,NaN,NaN,NaN
1,35455101,35454877,3,2017-01-01,03:34:00,3,2017-Ene,EME,466470-1,3526141-4,2017-01-01,2017-01-04,876,PAMI UGL VI,PAMI,1,RX,RAYOS,7100,VEDIA ANICETA,MEDICO DE PISO,34090503,RX C/PORTATIL DE TORAX,1,NaN,NaN,NaN,NaN
2,35455130,35454906,3,2017-01-01,04:26:00,4,2017-Ene,EME,466471-4,3714597-2,2017-01-01,2017-01-07,876,PAMI UGL VI,PAMI,0,RX,RAYOS,7100,DENNER GERMAN,MEDICO DE PISO,34090503,RX C/PORTATIL DE TORAX,1,NaN,NaN,NaN,NaN
3,35455135,35454911,3,2017-01-01,04:31:00,4,2017-Ene,EME,466473-0,3672378-8,2017-01-01,2017-01-04,948,OSUTHGRA O.S PERSONAL GASTRONOMICO,BRISTOL,1,RX,RAYOS,7100,CANIS DIEGO,MEDICO DE UCI,34042103,"ABDOMEN FRENTE, DE PIE Y ACOSTADO. RX DE",1,NaN,NaN,NaN,NaN
4,35455135,35454911,3,2017-01-01,04:31:00,4,2017-Ene,EME,466473-0,3672378-8,2017-01-01,2017-01-04,948,OSUTHGRA O.S PERSONAL GASTRONOMICO,BRISTOL,1,RX,RAYOS,7100,CANIS DIEGO,MEDICO DE UCI,34090503,RX C/PORTATIL DE TORAX,1,NaN,NaN,NaN,NaN


In [24]:
labos = pd.read_csv('SanatorioGüemes/MachLearning_Laboratorios_solicitados.csv', sep=";")


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
labos.head()

,Nro Vale,Pun,Estado,Fecha,Hora,HH,AñoMes,Sector,Nro Adm,HistClín,Nombre Paciente,Fec.Adm.,Fec.Alta,Entidad,Nombre Entidad,Nro.Afiliado,GrupoEnt,Urgencia,Mnemo Serv,Servicio,Cod Serv,Nombre del Solicitante,Función del Solicitante,Prestación,Descrip Prestación,CantPrest,Cod Insumo,Descrip Insumo,CantInsumos,EstadoResultado,Observ.Estudio
0,35455087,35454863,3,01/01/2017,03:20:00,3,2017-Ene,EME,466471-4,3714597-2,"HIDALGO,MARIA ANGELICA",01/01/2017,07/01/2017,876,PAMI UGL VI,15048873600400,PAMI,1,LA,LABORATORIO,7000,VEDIA ANICETA,MEDICO DE PISO,171,COAGULOGRAMA BASICO,1,NaN,NaN,NaN,NaN,NaN
1,35455087,35454863,3,01/01/2017,03:20:00,3,2017-Ene,EME,466471-4,3714597-2,"HIDALGO,MARIA ANGELICA",01/01/2017,07/01/2017,876,PAMI UGL VI,15048873600400,PAMI,1,LA,LABORATORIO,7000,VEDIA ANICETA,MEDICO DE PISO,190,CPK,1,NaN,NaN,NaN,NaN,NaN
2,35455087,35454863,3,01/01/2017,03:20:00,3,2017-Ene,EME,466471-4,3714597-2,"HIDALGO,MARIA ANGELICA",01/01/2017,07/01/2017,876,PAMI UGL VI,15048873600400,PAMI,1,LA,LABORATORIO,7000,VEDIA ANICETA,MEDICO DE PISO,412,GLUCOSA EN SANGRE,1,NaN,NaN,NaN,NaN,NaN
3,35455087,35454863,3,01/01/2017,03:20:00,3,2017-Ene,EME,466471-4,3714597-2,"HIDALGO,MARIA ANGELICA",01/01/2017,07/01/2017,876,PAMI UGL VI,15048873600400,PAMI,1,LA,LABORATORIO,7000,VEDIA ANICETA,MEDICO DE PISO,475,HEMOGRAMA,1,NaN,NaN,NaN,NaN,NaN
4,35455087,35454863,3,01/01/2017,03:20:00,3,2017-Ene,EME,466471-4,3714597-2,"HIDALGO,MARIA ANGELICA",01/01/2017,07/01/2017,876,PAMI UGL VI,15048873600400,PAMI,1,LA,LABORATORIO,7000,VEDIA ANICETA,MEDICO DE PISO,481,HEPATOGRAMA,1,NaN,NaN,NaN,NaN,NaN


In [26]:
labos.describe()


,Nro Vale,Pun,Estado,HH,Entidad,Urgencia,Cod Serv,Prestación,CantPrest,Cod Insumo,Descrip Insumo,CantInsumos,Observ.Estudio
count,3.086085e+06,3.086085e+06,3.086085e+06,3.086085e+06,3.086085e+06,3.086085e+06,3086085.0,3.086085e+06,3.086085e+06,0.0,0.0,0.0,0.0
mean,4.017415e+07,4.017385e+07,2.991985e+00,1.278370e+01,6.007930e+02,2.273810e-01,7000.0,1.152378e+03,1.000014e+00,NaN,NaN,NaN,NaN
std,2.612389e+06,2.612357e+06,8.941749e-02,5.487367e+00,3.924024e+02,4.191407e-01,0.0,1.876027e+03,7.887621e-03,NaN,NaN,NaN,NaN
min,3.545509e+07,3.545486e+07,1.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,7000.0,2.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN
25%,3.790430e+07,3.790403e+07,3.000000e+00,1.000000e+01,1.010000e+02,0.000000e+00,7000.0,4.120000e+02,1.000000e+00,NaN,NaN,NaN,NaN
50%,4.024358e+07,4.024327e+07,3.000000e+00,1.300000e+01,8.760000e+02,0.000000e+00,7000.0,5.460000e+02,1.000000e+00,NaN,NaN,NaN,NaN
75%,4.243598e+07,4.243566e+07,3.000000e+00,1.700000e+01,9.340000e+02,0.000000e+00,7000.0,9.020000e+02,1.000000e+00,NaN,NaN,NaN,NaN
max,4.454173e+07,4.454138e+07,3.000000e+00,2.300000e+01,9.930000e+02,1.000000e+00,7000.0,1.095400e+04,6.000000e+00,NaN,NaN,NaN,NaN


In [28]:
labos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3086085 entries, 0 to 3086084
Data columns (total 31 columns):
Nro Vale                   int64
Pun                        int64
Estado                     int64
Fecha                      object
Hora                       object
HH                         int64
AñoMes                     object
Sector                     object
Nro Adm                    object
HistClín                   object
Nombre Paciente            object
Fec.Adm.                   object
Fec.Alta                   object
Entidad                    int64
Nombre Entidad             object
Nro.Afiliado               object
GrupoEnt                   object
Urgencia                   int64
Mnemo Serv                 object
Servicio                   object
Cod Serv                   int64
Nombre del Solicitante     object
Función del Solicitante    object
Prestación                 int64
Descrip Prestación         object
CantPrest                  int64
Cod Insumo  